In [42]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
from datasets import *
import torch
import pandas as pd
import ast
import numpy as np

In [2]:
data_frame = pd.read_csv('../data/clean/sustainability-report-2020-clean-BST_KK-fixed-with_starts.csv', sep=';')

In [3]:
# Combine answe and answer_start into a new column called answers, make it a dict with keys 'text' and 'answer_start'
data_frame['answers'] = data_frame.apply(lambda x: {'text': [x['answer']], 'answer_start': [x['answer_start']]}, axis=1)

In [4]:
# Add id column
data_frame['id'] = data_frame.index

In [5]:
# Drop column fLABEl
data_frame = data_frame.drop(columns=['fLABEL', "answer", "answer_start"])

In [6]:
data_frame

,question,context,answers,id
0,What percentage of female managers were women ...,nts.56% of female managersAt the end of the ye...,"{'text': ['56%'], 'answer_start': [4]}",0
1,What percentage of managers were women in the ...,"e end of the year, women represented56% of all...","{'text': ['49%'], 'answer_start': [69]}",1
2,How much of managers in the second management ...,nagement positions. 49% ofwomen at the first m...,"{'text': ['55%'], 'answer_start': [67]}",2
3,What percentage of Supervisory and Management ...,Women represent 31% of allSupervisory and Mana...,"{'text': ['31%'], 'answer_start': [16]}",3
4,"In 2020, what was the proportion of staff with...",rtunities. In 2020 the proportionof staff with...,"{'text': ['2%.'], 'answer_start': [69]}",4
...,...,...,...,...
180,What percentage of Komercijalna Banka a.d. Beo...,of work and creativity.With the completion of...,"{'text': ['83.23%'], 'answer_start': [69]}",180
181,When was Banka a.d. Beograd acquired?,ry shareholding inKomercijalna Banka a.d. Beog...,"{'text': ['2020'], 'answer_start': [67]}",181
182,What is the name of the financial institution ...,e completion of the acquisition of an 83.23% o...,"{'text': ['Komercijalna Banka a.d. Beograd'], ...",182
183,When was Komercijalna Banka acquired?,"in our home, the SEE region.Since Komercijalna...","{'text': ['30th December 2020'], 'answer_start...",183


In [7]:
data_frame.to_csv('../data/clean/sustainability-report-2020-squad-format.csv', sep=';', index=False)

In [8]:
data = load_dataset('csv', data_files="../data/clean/sustainability-report-2020-squad-format.csv", delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/rjutr/.cache/huggingface/datasets/csv/default-6dc907f97847b38a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [9]:
data["test"] = data["test"].map(lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {"text": example["text"], "answer_start": example["answer_start"]}})
data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {"text": example["text"], "answer_start": example["answer_start"]}})
data["train"].remove_columns(["text", "answer_start"])

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'context', 'answers', 'id'],
    num_rows: 129
})

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [11]:
def tokenize_sample_data(data):
  # tokenize
  tokenized_feature = tokenizer(
    data["question"],
    data["context"],
    max_length = 384,
    return_overflowing_tokens=True,
    stride=128,
    truncation="only_second",
    padding = "max_length",
    return_offsets_mapping=True,
  )

  # When it overflows, multiple rows will be returned for a single example.
  # The following then gets the array of corresponding the original sample index.
  sample_mapping = tokenized_feature.pop("overflow_to_sample_mapping")
  # Get the array of [start_char, end_char + 1] in each token.
  # The shape is [returned_row_size, max_length]
  offset_mapping = tokenized_feature.pop("offset_mapping")

  start_positions = []
  end_positions = []
  for i, offset in enumerate(offset_mapping):
    sample_index = sample_mapping[i]
    answers = data["answers"][sample_index]
    start_char = answers["answer_start"][0]
    end_char = start_char + len(answers["text"][0]) - 1
    # The format of sequence_ids is [None, 0, ..., 0, None, None, 1, ..., 1, None, None, ...]
    # in which question's token is 0 and contex's token is 1
    sequence_ids = tokenized_feature.sequence_ids(i)
    # find the start and end index of context
    idx = 0
    while sequence_ids[idx] != 1:
      idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1
    # Set start positions and end positions in inputs_ids
    # Note: The second element in offset is end_char + 1
    #if offset[context_start][0] > end_char or offset[context_end][1] <= start_char:
    if not (offset[context_start][0] <= start_char and end_char < offset[context_end][1]):
      # The case that answer is not inside the context
      ## Note : Some tokenizer (such as, tokenizer in rinna model) doesn't place CLS
      ## for the first token in sequence, and I then set -1 as positions.
      ## (Later I'll process rows with start_positions=-1.)
      start_positions.append(-1)
      end_positions.append(-1)
    else:
      # The case that answer is found in the context

      # Set start position
      idx = context_start
      while offset[idx][0] < start_char:
        idx += 1
      if offset[idx][0] == start_char:
        start_positions.append(idx)
      else:
        start_positions.append(idx - 1)

      # Set end position
      idx = context_end
      while offset[idx][1] > end_char + 1:
        idx -= 1
      if offset[idx][1] == end_char + 1:
        end_positions.append(idx)
      else:
        end_positions.append(idx + 1)

  # build result
  tokenized_feature["start_positions"] = start_positions
  tokenized_feature["end_positions"] = end_positions   
  return tokenized_feature

# Run conversion
tokenized_ds = data.map(
  tokenize_sample_data,
  remove_columns=["id", "context", "question", "answers"],
  batched=True,
  batch_size=128)

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [12]:
tokenized_ds = tokenized_ds.filter(lambda x: x["start_positions"] != -1)

Filter:   0%|          | 0/129 [00:00<?, ? examples/s]

Filter:   0%|          | 0/56 [00:00<?, ? examples/s]

In [13]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'answer_start', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 129
    })
    test: Dataset({
        features: ['text', 'answer_start', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 56
    })
})

In [14]:
from transformers import AutoConfig, AutoModelForQuestionAnswering

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = (AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased").to(device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [15]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir = "distilbert-nlb-qa",
  log_level = "error",
  num_train_epochs = 200,
  learning_rate = 7e-5,
  lr_scheduler_type = "linear",
  warmup_steps = 100,
  per_device_train_batch_size = 16,
  per_device_eval_batch_size = 16,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 150,
  save_steps = 500,
  logging_steps = 50,
  push_to_hub = False
)

In [28]:
from transformers import Trainer

trainer = Trainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  train_dataset = tokenized_ds["train"],
  eval_dataset = tokenized_ds["test"].select(range(data["test"].shape[0] // 2)),
  tokenizer = tokenizer,
)

trainer.train()

c:\Users\rjutr\miniconda3\envs\project_ds\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 1.6773, 'learning_rate': 3.5e-05, 'epoch': 33.78}
{'loss': 0.0683, 'learning_rate': 7e-05, 'epoch': 67.0}
{'loss': 0.0016, 'learning_rate': 3.5e-05, 'epoch': 100.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 3.0335118770599365, 'eval_runtime': 0.1735, 'eval_samples_per_second': 161.402, 'eval_steps_per_second': 11.529, 'epoch': 100.0}
{'loss': 0.0005, 'learning_rate': 0.0, 'epoch': 133.33}
{'train_runtime': 306.5086, 'train_samples_per_second': 84.174, 'train_steps_per_second': 0.653, 'train_loss': 0.4369335945695639, 'epoch': 133.33}


TrainOutput(global_step=200, training_loss=0.4369335945695639, metrics={'train_runtime': 306.5086, 'train_samples_per_second': 84.174, 'train_steps_per_second': 0.653, 'train_loss': 0.4369335945695639, 'epoch': 133.33})

In [31]:
import torch
import numpy as np

def inference_answer(question, context):
  question = question
  context = context
  test_feature = tokenizer(
    question,
    context,
    max_length=318,
  )
  with torch.no_grad():
    outputs = model(torch.tensor([test_feature["input_ids"]]).to(device))
  start_logits = outputs.start_logits.cpu().numpy()
  end_logits = outputs.end_logits.cpu().numpy()
  answer_ids = test_feature["input_ids"][np.argmax(start_logits):np.argmax(end_logits)+1]
  return "".join(tokenizer.batch_decode(answer_ids))


answer_pred = [inference_answer(data["test"]["question"][idx], data["test"]["context"][idx]) for idx in range(data["test"].shape[0])]

In [32]:
answer_true = [data["test"]["answers"][idx]["text"][0] for idx in range(data["test"].shape[0])]

In [48]:
import evaluate
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=answer_pred, references=answer_true, lang="en")

In [49]:
# mean all of the metrics
print(f"F1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

F1: 0.78750842916114, Precision: 0.7792423899684634, Recall: 0.7983093421374049


In [109]:
references = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in zip(data["test"]["id"], data["test"]["answers"])]

In [110]:
predictions = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data["test"]["id"], answer_pred)]

In [111]:
squad_v2_metric = evaluate.load("squad_v2")
results = squad_v2_metric.compute(predictions=predictions, references=references)
results

{'exact': 30.357142857142858,
 'f1': 31.944444444444446,
 'total': 56,
 'HasAns_exact': 30.357142857142858,
 'HasAns_f1': 31.944444444444446,
 'HasAns_total': 56,
 'best_exact': 30.357142857142858,
 'best_exact_thresh': 0.0,
 'best_f1': 31.944444444444446,
 'best_f1_thresh': 0.0}